In [56]:
import numpy as np
import pandas as pd
import json

In [34]:
data = pd.read_csv('http://taxbills.nyc/joined.csv',
                   usecols = ['borough', 'ucbbl', '2007uc',
                              '2008uc', '2009uc', '2010uc',
                              '2011uc', '2012uc', '2013uc',
                              '2014uc', 'cd', 'zipcode',
                              'unitsres'])
    
#
# Make a new row for each year
#
mdata = pd.melt(data, id_vars = ['borough', 'ucbbl', 'cd', 'zipcode',
                                 'unitsres'])
mdata.rename(columns = {'variable': 'year',
                        'value': 'Nstabilized'}, inplace = True)
mdata['year'] = mdata.apply(lambda x: np.int64(x.year[0:4]), axis = 1)

    
#
# Get rid of location identifiers that you're not interested in
#     for now
#
mdata.drop(['borough', 'ucbbl', 'cd'], axis = 1,
            inplace = True)
     
mdata = mdata.groupby(['zipcode', 'year']).sum().reset_index()

mdata['Pstabilized'] = mdata['Nstabilized'] / mdata['unitsres']


mdata.drop(['unitsres'], axis = 1, inplace = True)

In [35]:
mdata.head()

,zipcode,year,Nstabilized,Pstabilized
0,10001,2007,5195,0.532329
1,10001,2008,5222,0.535096
2,10001,2009,4703,0.481914
3,10001,2010,4628,0.474229
4,10001,2011,4835,0.495440


In [36]:
  
#
# Drop values with NA zipcodes
#
print(mdata.shape)
mdata.dropna(inplace = True)
print(mdata.shape)
mdata['zipcode'] = mdata['zipcode'].astype(int)
mdata['Nstabilized'] = mdata['Nstabilized'].astype(int)
print(mdata.dtypes)

#
# Save to CSV
#
mdata.to_csv('stabilized.csv', index = False)
    

(1416, 4)
(1406, 4)
zipcode          int64
year             int64
Nstabilized      int64
Pstabilized    float64
dtype: object


In [37]:
mdata.head()

,zipcode,year,Nstabilized,Pstabilized
0,10001,2007,5195,0.532329
1,10001,2008,5222,0.535096
2,10001,2009,4703,0.481914
3,10001,2010,4628,0.474229
4,10001,2011,4835,0.495440


In [38]:
data = pd.read_csv('data/311_Heat_Seek_Subset.csv', 
                   usecols = [1, 5, 8], header = None)
data.rename(columns = {1: 'year', 5: 'complaint_type', 8: 'zipcode'}, inplace = True)
data['year'] = data['year'].apply(lambda x: x[6:10])
data = data.groupby(['zipcode', 'year']).count().reset_index()
data.rename(columns = {'complaint_type': 'complaint_count'}, inplace = True)

data['year'] = data['year'].astype(np.int64)
data['zipcode'] = data['zipcode'].astype(np.int64)

data.head()

,zipcode,year,complaint_count
0,10001,2010,648
1,10001,2011,468
2,10001,2012,344
3,10001,2013,289
4,10001,2014,437


In [39]:
print(data.dtypes)
print(mdata.dtypes)

zipcode            int64
year               int64
complaint_count    int64
dtype: object
zipcode          int64
year             int64
Nstabilized      int64
Pstabilized    float64
dtype: object


In [40]:
merge_data = pd.merge(data, mdata, how = 'inner', on = ['zipcode', 'year'])
merge_data.head()

,zipcode,year,complaint_count,Nstabilized,Pstabilized
0,10001,2010,648,4628,0.474229
1,10001,2011,468,4835,0.495440
2,10001,2012,344,5514,0.565017
3,10001,2013,289,5228,0.535711
4,10001,2014,437,5032,0.515627


In [71]:
ind = np.where(np.isinf(merge_data['Pstabilized']))
ind
merge_data.iloc[255]
merge_data.drop(merge_data.index[np.where(np.isinf(merge_data['Pstabilized']))], inplace = True)
ind = np.where(np.isinf(merge_data['Pstabilized']))
ind

(array([], dtype=int64),)

In [72]:
merge_data.to_csv('jointdata.csv')

In [73]:
output_json = {}
output_json['type'] = 'zipcode_data_collection'
output_json['features'] = []

for zc in np.unique(merge_data['zipcode'].values):
    entry = {}
    entry['zipcode'] = zc
    
    entry['data'] = []
    hold = merge_data[merge_data.zipcode == zc]
    for ind, row in hold.iterrows():
        data_entry = {}
        data_entry['year'] = row['year']
        data_entry['complaint_count'] = row['complaint_count']
        data_entry['percent_stabilized'] = row['Pstabilized']
        data_entry['number_stabilized'] = row['Nstabilized']
        entry['data'].append(data_entry)
        
    output_json['features'].append(entry)

In [74]:
with open('jointdata.json', 'w') as f:
    json.dump(output_json, f)